Import Libraries

In [1]:
%matplotlib inline
import matplotlib.pylab as plt 
from matplotlib import cm

import os
import math
import numpy as np
import pandas as pd
import cma
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, Callback
import tensorflow.keras.backend as K
from IPython.display import clear_output
from time import time, gmtime, strftime, localtime

import warnings
warnings.filterwarnings("ignore")

2022-12-14 13:07:16.191107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from scipy import dot, exp, log, sqrt, floor, ones, randn, zeros_like, Inf, argmax, argmin, eye, outer, zeros
from scipy import array, power, diag, cos, sin, pi
from scipy.linalg import expm, cholesky, solve, norm, eig
from scipy.stats import multivariate_normal

import seaborn as sns

## run on CPU...
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# disable eager excution
tf.compat.v1.disable_eager_execution()



### 1. PINN for kdv equation

#### 1.0. Data

#### 1.1. Geometry, PDE parameter & BC

In [3]:
vis = 0.001

# initial condition
k = 2

# spatial domain
x_l, x_u = -2.0, 2.0

# time domain: 0 - t_T
t_T = 2.0
# function to generate IC
def f_ic(x):
    return np.exp(-(k*x)**2)

#### 1.2. Sampling plan

In [4]:
sim = pd.read_csv('three.csv')
sim['x'], sim['t'] = sim['x'], sim['t']
batch_X = np.vstack([sim.t.values, sim.x.values]).T
y_train = sim[['u']].values
# y_train.shape

#### 1.3. PDE-NN

In [5]:
# specify NN model + Physis
def create_nn(nodes):

    # input layers -> split into (t, x, z)
    inputs = layers.Input(shape=(2,))
    t, x = layers.Lambda( lambda k: tf.split(k, num_or_size_splits=2, axis=1))(inputs)

    # hidden layers
    hidden_1 = layers.Dense(nodes, activation='tanh')(layers.Concatenate()([t, x]))
    hidden_2 = layers.Dense(nodes, activation='tanh')(hidden_1)
    hidden_l = layers.Dense(nodes, activation='tanh')(hidden_2)

    # output layers
    u = layers.Dense(1, name="u", use_bias=False)(hidden_l)

    # axillary PDE outputs
    u_t = K.gradients(u, t)[0]
    u_x = K.gradients(u, x)[0]
    u_xx = K.gradients(u_x, x)[0]
    pde_residuals = u_t + u*u_x - vis*u_xx

    # PDE residuals (u_t + u*u_x - vis*u_xx = 0)
    pde_mse = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(pde_residuals), predictions=pde_residuals)

    # initial conditions: u(0, x) = exp(-(k*x)**2)
    x_0, u_0 = tf.boolean_mask(x, tf.equal(t, 0)), tf.boolean_mask(u, tf.equal(t, 0))
    ic = tf.math.exp(tf.math.negative(tf.math.square(k*x_0)))
    ic_mse = tf.compat.v1.losses.mean_squared_error(labels=ic, predictions=u_0)


    # initiate model
    nn = models.Model(inputs=inputs, outputs=u)

    # optimizer
    optimizer = tf.keras.optimizers.Adam(0.05)

    # compile model with [physics] loss
    nn.compile(loss = compute_physic_loss(pde_mse, ic_mse),
               optimizer = optimizer,
               metrics = [compute_pde_loss(pde_mse), compute_ic_loss(ic_mse)])

    # pathway to variables inside NN
    insiders = [pde_mse, ic_mse]
    eval_ins = K.function([nn.input, K.learning_phase()], insiders)   # evaluation function

    return (nn, eval_ins)

In [6]:
# define loss function (PDE + IC loss)
def compute_physic_loss(pde_mse, ic_mse):
    def physic_loss(y_true, y_pred):
        pde_loss  = pde_mse # PDE
        ic_loss   = ic_mse  # IC
        return pde_loss + ic_loss
    return physic_loss

# define loss function (PDE loss)
def compute_pde_loss(pde_mse):
    def pde_loss(y_true, y_pred): return pde_mse
    return pde_loss # return a function

# define loss function (IC loss)
def compute_ic_loss(ic_mse):
    def ic_loss(y_true, y_pred): return ic_mse
    return ic_loss # return a function

# define loss function (data loss)
def compute_data_loss(z):
    def data_loss(y_true, y_pred):
        p_data = tf.equal(z, 1)
        return tf.losses.mean_squared_error(labels=tf.boolean_mask(y_true, p_data), predictions=tf.boolean_mask(y_pred, p_data))
    return data_loss

### 3. Optimize PINN with [Neuroevolution / xNES]

In [7]:
# initiate NN model (& pathway to internal values)
n_nodes = 8
nn, eval_ins = create_nn(n_nodes)

# initial weights
w0 = np.array([])
nn_weights = nn.get_weights()
for _g in nn_weights: w0 = np.append(w0, _g.flatten())

nn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 [(None, 1), (None, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           lambda[0][0]                     
                                                                 lambda[0][1]                     
__________________________________________________________________________________________________
dense (Dense)                   (None, 8)            24          concatenate[0][0]            

2022-12-14 13:07:19.756814: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-14 13:07:19.799659: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-14 13:07:19.799694: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ihpcgs-03
2022-12-14 13:07:19.799700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ihpcgs-03
2022-12-14 13:07:19.799821: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-12-14 13:07:19.799846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-12-14 13:07:19.799851: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-12-14 13:07:19.800278: I tensorflow/core/platform/cpu_fe

In [8]:
# nn weights structure 
nn_weights = nn.get_weights()
nn_wlen = len(nn_weights)
nn_wshape = [_w.shape for _w in nn_weights]
nn_widx = np.cumsum([len(_w.flatten()) for _w in nn_weights])
nn_nweights = nn_widx[-1]

In [9]:

def nn_fitness(_weights):
    _weights = np.split(_weights, nn_widx[:-1])
    _weights = [_weights[i].reshape(nn_wshape[i]) for i in range(nn_wlen)]
    nn.set_weights(_weights)
    _x = batch_X
    pde_mse, ic_mse = eval_ins(_x)
    fitness = (pde_mse + ic_mse)
    return fitness

In [10]:
x, es = cma.fmin2(nn_fitness, w0, 1e-2, options={'ftarget':1e-5, 'popsize':100, 'maxiter':250, 'verb_disp':1})

(50_w,100)-aCMA-ES (mu_w=27.0,w_1=8%) in dimension 176 (seed=1171944, Wed Dec 14 13:07:19 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    100 1.920512318611145e-01 1.0e+00 9.51e-03  9e-03  1e-02 0:01.0
    2    200 1.799505352973938e-01 1.0e+00 9.35e-03  9e-03  9e-03 0:01.8
    3    300 1.662152707576752e-01 1.0e+00 9.34e-03  9e-03  9e-03 0:02.6
    4    400 1.573499888181686e-01 1.0e+00 9.56e-03  1e-02  1e-02 0:03.4
    5    500 1.484290659427643e-01 1.0e+00 9.94e-03  1e-02  1e-02 0:04.2
    6    600 1.422898471355438e-01 1.0e+00 1.05e-02  1e-02  1e-02 0:05.0
    7    700 1.317244768142700e-01 1.0e+00 1.12e-02  1e-02  1e-02 0:05.8
    8    800 1.261587738990784e-01 1.0e+00 1.20e-02  1e-02  1e-02 0:06.6
    9    900 1.169941276311874e-01 1.0e+00 1.30e-02  1e-02  1e-02 0:07.4
   10   1000 1.119214892387390e-01 1.0e+00 1.42e-02  1e-02  1e-02 0:08.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11   1100 1.094646230340004

  102  10200 2.303650230169296e-02 1.4e+00 1.12e-01  1e-01  1e-01 1:21.9
  103  10300 1.997710391879082e-02 1.4e+00 1.11e-01  1e-01  1e-01 1:22.7
  104  10400 2.107880078256130e-02 1.4e+00 1.10e-01  1e-01  1e-01 1:23.5
  105  10500 1.721407845616341e-02 1.4e+00 1.09e-01  1e-01  1e-01 1:24.3
  106  10600 1.990635693073273e-02 1.4e+00 1.09e-01  1e-01  1e-01 1:25.1
  107  10700 1.665426790714264e-02 1.5e+00 1.10e-01  1e-01  1e-01 1:25.8
  108  10800 1.679712347686291e-02 1.5e+00 1.10e-01  1e-01  1e-01 1:26.6
  109  10900 1.822420582175255e-02 1.5e+00 1.10e-01  1e-01  1e-01 1:27.4
  110  11000 1.901497505605221e-02 1.5e+00 1.11e-01  1e-01  1e-01 1:28.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  111  11100 1.730090565979481e-02 1.5e+00 1.12e-01  1e-01  1e-01 1:29.0
  112  11200 2.018042095005512e-02 1.5e+00 1.13e-01  1e-01  1e-01 1:29.8
  113  11300 1.799843832850456e-02 1.5e+00 1.13e-01  1e-01  1e-01 1:30.6
  114  11400 1.914010010659695e-02 1.5e+00 1.14e-01 

  205  20500 1.487403735518456e-02 1.7e+00 1.03e-01  9e-02  1e-01 2:43.9
  206  20600 1.608620211482048e-02 1.8e+00 1.02e-01  9e-02  1e-01 2:44.7
  207  20700 1.266595721244812e-02 1.8e+00 1.01e-01  9e-02  1e-01 2:45.5
  208  20800 1.322276331484318e-02 1.8e+00 1.00e-01  9e-02  1e-01 2:46.3
  209  20900 1.312277000397444e-02 1.8e+00 9.86e-02  9e-02  1e-01 2:47.0
  210  21000 1.417652331292629e-02 1.8e+00 9.79e-02  9e-02  1e-01 2:47.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  211  21100 1.303756795823574e-02 1.8e+00 9.73e-02  9e-02  1e-01 2:48.7
  212  21200 1.320992968976498e-02 1.8e+00 9.75e-02  9e-02  1e-01 2:49.5
  213  21300 1.464144326746464e-02 1.8e+00 9.72e-02  9e-02  1e-01 2:50.3
  214  21400 1.216378621757030e-02 1.8e+00 9.69e-02  9e-02  1e-01 2:51.1
  215  21500 1.229878515005112e-02 1.8e+00 9.61e-02  9e-02  1e-01 2:51.9
  216  21600 1.238964218646288e-02 1.8e+00 9.61e-02  9e-02  1e-01 2:52.6
  217  21700 1.397662516683340e-02 1.8e+00 9.60e-02 